In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from sparsesvd import sparsesvd
import scipy
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import *
from sklearn.model_selection import train_test_split
%matplotlib inline

In [22]:
#ratings input (taking only the first three columns)
ratings_columns = ['userid', 'movieid', 'rating']
ratings = pd.read_csv('./ml-100k/u.data', sep = '\t', names = ratings_columns, usecols = range(3))

In [23]:
#movie inputs (Taking 2 columns)
movie_columns = ['movieid', 'title']
movies = pd.read_csv('./ml-100k/u.item', sep = '|', engine = 'python', names = movie_columns, usecols= range(2))

In [124]:
ratings.head()

,userid,movieid,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [125]:
movies.head()

,movieid,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [126]:
#merging both dataframes
dataset = pd.merge(ratings, movies)

In [127]:
dataset.head()

,userid,movieid,rating,title
0,196,242,3,Kolya (1996)
1,63,242,3,Kolya (1996)
2,226,242,5,Kolya (1996)
3,154,242,3,Kolya (1996)
4,306,242,5,Kolya (1996)


In [130]:
#Pivoting table to cerate a user-item matrix
user_movies = dataset.pivot_table(index = 'userid', columns= 'title', values = 'rating')

In [131]:
user_movies.head()

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,� k�ldum klaka (Cold Fever) (1994)
userid,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,2.0,5.0,NaN,NaN,3.0,4.0,NaN,NaN,...,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,2.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,4.0,NaN


In [430]:
#creating a item-item matrix to check similarity in ratings between 2 movies
corr_matrix = user_movies.corr()

In [431]:
corr_matrix.head()

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,� k�ldum klaka (Cold Fever) (1994)
title,,,,,,,,,,,,,,,,,,,,,
'Til There Was You (1997),1.0,NaN,-1.000000,-0.500000,-0.500000,0.522233,NaN,-0.426401,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1-900 (1994),NaN,1.0,NaN,NaN,NaN,NaN,NaN,-0.981981,NaN,NaN,...,NaN,NaN,NaN,-0.944911,NaN,NaN,NaN,NaN,NaN,NaN
101 Dalmatians (1996),-1.0,NaN,1.000000,-0.049890,0.269191,0.048973,0.266928,-0.043407,NaN,0.111111,...,NaN,-1.000000,NaN,0.158840,0.119234,0.680414,0.000000,0.707107,NaN,NaN
12 Angry Men (1957),-0.5,NaN,-0.049890,1.000000,0.666667,0.256625,0.274772,0.178848,NaN,0.457176,...,NaN,NaN,NaN,0.096546,0.068944,-0.361961,0.144338,1.000000,1.0,NaN
187 (1997),-0.5,NaN,0.269191,0.666667,1.000000,0.596644,NaN,-0.554700,NaN,1.000000,...,NaN,0.866025,NaN,0.455233,-0.500000,0.500000,0.475327,NaN,NaN,NaN


In [433]:
#Looking to see star wars similarities with other movies
corr_matrix.loc['Star Wars (1977)'].sort_values(ascending = False)

title
Man of the Year (1995)                                                               1.000000
Full Speed (1996)                                                                    1.000000
Star Wars (1977)                                                                     1.000000
Mondo (1996)                                                                         1.000000
Line King: Al Hirschfeld, The (1996)                                                 1.000000
Outlaw, The (1943)                                                                   1.000000
Hurricane Streets (1998)                                                             1.000000
Hollow Reed (1996)                                                                   1.000000
Scarlet Letter, The (1926)                                                           1.000000
Safe Passage (1994)                                                                  1.000000
Good Man in Africa, A (1994)                          




What was interesting to see here is that movies like Full Speed and Man of the year (which I haven't even heard of) were highly correlated with Star Wars. This didn't make sense to me, so I decided to look at how many times such movies were rated.

In [483]:
dataset[dataset['title'] == "Full Speed (1996)"]

,userid,movieid,rating,title
99120,100,1238,2,Full Speed (1996)
99121,489,1238,4,Full Speed (1996)
99122,404,1238,3,Full Speed (1996)
99123,423,1238,3,Full Speed (1996)
99124,519,1238,5,Full Speed (1996)
99125,512,1238,4,Full Speed (1996)
99126,845,1238,2,Full Speed (1996)
99127,655,1238,2,Full Speed (1996)


As we can see, Full speed was only rated 8 times and rated reasonably well, therefore seemed to be closer to a movie as legendary as Star Wars. So I decided to improve on the correlation matrix by looking at relationships where both movies were rated atleast 90 times.

In [434]:
corr_matrix = user_movies.corr(method = 'pearson', min_periods = 90)

In [435]:
corr_matrix.head()

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,� k�ldum klaka (Cold Fever) (1994)
title,,,,,,,,,,,,,,,,,,,,,
'Til There Was You (1997),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1-900 (1994),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101 Dalmatians (1996),NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12 Angry Men (1957),NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
187 (1997),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now lets look at movies that are closely related to Star Wars

In [436]:
corr_matrix.loc['Star Wars (1977)'].sort_values(ascending = False)

title
Star Wars (1977)                                            1.000000
Empire Strikes Back, The (1980)                             0.747981
Return of the Jedi (1983)                                   0.672556
Raiders of the Lost Ark (1981)                              0.536117
Austin Powers: International Man of Mystery (1997)          0.377433
Sting, The (1973)                                           0.367538
Indiana Jones and the Last Crusade (1989)                   0.350107
Pinocchio (1940)                                            0.347868
Frighteners, The (1996)                                     0.332729
L.A. Confidential (1997)                                    0.319065
Dumbo (1941)                                                0.317656
Bridge on the River Kwai, The (1957)                        0.316580
Miracle on 34th Street (1994)                               0.310921
E.T. the Extra-Terrestrial (1982)                           0.303619
Mystery Science Theater 3000

Now we see a clearer picture of movies that are closely related to star wars based on ratings. The other Star Wars movies appear on the list. 

In [446]:
#creating a dictionary of penalties where movies that are rated highly are given additional bias and 
#movies rated poorly are given penalties

penalties = {1: -20, 2: -10, 3: 0, 4: 10, 5: 20}

In [465]:
#creating a function to fetch the user
def fetch_user(dataframe, index):
    
    return dataframe.iloc[index].dropna()

#getting the recommendations
def get_recommendations(user):
    recommendations = pd.Series()
    for i in range(len(user)):
        similarity = corr_matrix[user.index[i]].dropna()
        similarity = similarity.map(lambda x: (x * user[i]** 2) + (5 * penalties[user[i]]))
        recommendations = recommendations.append(similarity)
        
    recommendations = recommendations.groupby(recommendations.index).sum()
    recommendations.sort_values(ascending = False, inplace= True)
    return recommendations

In [466]:
#function to get the movie recommendations by inputting the user removing movies already watched by the user
def recommender(dataframe, index):
    user = fetch_user(dataframe, index)
    movie_recommend = get_recommendations(user)
    for movie in movie_recommend.index:
        if movie in user.index:
            movie_recommend.drop(movie, inplace = True)
    return movie_recommend

In [473]:
#creating a new user based on his love for adventure and fantasy movies and hatred for rom-coms
user_mov = pd.DataFrame({'Empire Strikes Back, The (1980)': 5, 'Pretty Woman (1990)' : 5, 'True Romance (1993)' : 1,
                        'Return of the Jedi (1983)' : 5, 'Kiss the Girls (1997)': 1, 'Sleepless in Seattle (1993)': 1,
                        'Star Trek: First Contact (1996)': 5, 'First Wives Club, The (1996)': 1,
                        'Star Wars (1977)' : 5}, index = [0])
user_mov.index.name = 'user'

In [474]:
user_mov

,"Empire Strikes Back, The (1980)","First Wives Club, The (1996)",Kiss the Girls (1997),Pretty Woman (1990),Return of the Jedi (1983),Sleepless in Seattle (1993),Star Trek: First Contact (1996),Star Wars (1977),True Romance (1993)
user,,,,,,,,,
0,5,1,1,5,5,1,5,5,1


In [475]:
#getting the recommendations for the user
recommender(user_mov, 0)

Back to the Future (1985)                        445.615886
Indiana Jones and the Last Crusade (1989)        445.086922
True Lies (1994)                                 441.007040
Dave (1993)                                      440.386217
Star Trek: The Wrath of Khan (1982)              439.867944
Batman (1989)                                    438.397599
Braveheart (1995)                                438.325858
Star Trek III: The Search for Spock (1984)       437.609341
Fugitive, The (1993)                             437.584467
Jurassic Park (1993)                             437.288756
Field of Dreams (1989)                           436.821662
Mrs. Doubtfire (1993)                            436.391946
Star Trek IV: The Voyage Home (1986)             436.050547
Aladdin (1992)                                   435.793739
Star Trek: Generations (1994)                    434.872092
Terminator 2: Judgment Day (1991)                434.192705
Ghost (1990)                            

We can see from the set of recommendations above that a lot of similar movies show up such as the other star trek movies, Indiana Jones and Back to the Future. Star Trek Generations seemed to quite low on the list, considering the user liked Star Trek: First Contact, but on seeing that Generations had performed poorly, it made more sense

Creating a recommender using matrix decomposition

In [524]:
user_movies.head()

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,� k�ldum klaka (Cold Fever) (1994)
userid,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,2.0,5.0,NaN,NaN,3.0,4.0,NaN,NaN,...,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,2.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,4.0,NaN


In [862]:
# Taking the user movie matrix and filling nans with 0s.
svd_movies = user_movies.fillna(0)
svd_movies.head()

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,� k�ldum klaka (Cold Fever) (1994)
userid,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,2.0,5.0,0.0,0.0,3.0,4.0,0.0,0.0,...,0.0,0.0,0.0,5.0,3.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,2.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0


In [863]:
#Dividing each rating a user gave by the mean of each user's rating
user_means = np.array(svd_movies.mean(axis = 1)).reshape(-1, 1)
svd_movies = svd_movies.div(svd_movies.mean(axis = 1), axis = 0)
svd_movies_matrix = svd_movies.as_matrix()

In [864]:
user_means

array([[ 0.58713942],
       [ 0.13581731],
       [ 0.08834135],
       [ 0.0625    ],
       [ 0.30228365],
       [ 0.45492788],
       [ 0.95492788],
       [ 0.13461538],
       [ 0.05649038],
       [ 0.46514423],
       [ 0.37680288],
       [ 0.13461538],
       [ 1.17698317],
       [ 0.24098558],
       [ 0.17788462],
       [ 0.36418269],
       [ 0.05108173],
       [ 0.64603365],
       [ 0.04266827],
       [ 0.08954327],
       [ 0.28575721],
       [ 0.25721154],
       [ 0.32992788],
       [ 0.17668269],
       [ 0.18990385],
       [ 0.18930288],
       [ 0.04867788],
       [ 0.17427885],
       [ 0.07451923],
       [ 0.09735577],
       [ 0.08473558],
       [ 0.07902644],
       [ 0.05108173],
       [ 0.04867788],
       [ 0.04176683],
       [ 0.04567308],
       [ 0.125     ],
       [ 0.27043269],
       [ 0.04987981],
       [ 0.06069712],
       [ 0.11778846],
       [ 0.40985577],
       [ 0.49278846],
       [ 0.33112981],
       [ 0.10396635],
       [ 0

In [ ]:
###Singular Value Decomposition

What SVD does is, it takes a matrix and gives three matrices as the output -> 'U' (orthogonal), 'S'(diagonal) and 'Vt'(transpose of an orthogonal). SVD is a matrix factorization technique that takes a matrix and decomposes it to the best lower rank(takes the best features, kind of like pca). In this case U is the users feature matrix (something like how much users "like" certain features), Vt is the items vector and S is the diagonal matrix of singular values (something like weights).

In [877]:
#getting the U, S and Vt values
U, sigma, Vt = svds(svd_movies_matrix, k = 10)

In [878]:
#Sigma value above is outputed as an array, but we need it in the form of a diagonal matrix
sigma = np.diag(sigma)

In [889]:
#creating predictions
predicted = np.dot(np.dot(U, sigma), Vt)
predicted_ratings = np.dot(np.dot(U, sigma), Vt) * user_means

In [890]:
predicted_ratings

array([[  4.13204703e-02,   2.86849575e-02,   6.62762788e-01, ...,
          2.71406035e-02,   6.36375179e-02,   1.12078002e-03],
       [  4.36798221e-03,   4.92118098e-02,   2.76978840e-01, ...,
          1.80681660e-02,  -3.72138894e-03,   1.56169105e-04],
       [ -1.35259823e-03,   1.38577574e-04,  -6.50964488e-02, ...,
          4.88776468e-04,   6.64276776e-02,   4.31446527e-05],
       ..., 
       [ -1.54807292e-03,   2.53200421e-02,   2.33365803e-01, ...,
          8.15698056e-03,   3.27823939e-03,  -6.74439125e-05],
       [  2.72674392e-02,  -2.54708924e-02,   2.11709355e-01, ...,
          2.87289514e-04,  -8.85277957e-03,   3.62645916e-04],
       [  3.24977410e-02,  -3.86551318e-02,   4.96014599e-01, ...,
          3.67927251e-02,   5.64052327e-02,   7.08069591e-04]])

In [870]:
predicted_df = pd.DataFrame(predicted_ratings, columns= user_movies.columns)
#Data frame index starts with 0 but original dataset starts with 1, so adding 1 to index
predicted_df.index = predicted_df.index + 1

In [871]:
predicted_df.head()

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,� k�ldum klaka (Cold Fever) (1994)
1,0.041320,0.028685,0.662763,1.618436,0.062006,0.596432,0.493691,3.074301,0.029426,0.654572,...,0.000742,0.036697,0.003545,2.302366,0.616471,0.211703,0.527240,0.027141,0.063638,0.001121
2,0.004368,0.049212,0.276979,0.208428,-0.094744,0.298444,-0.035093,0.003464,0.028983,0.070455,...,-0.000120,0.027676,-0.000288,-0.060491,-0.086959,-0.036478,0.243128,0.018068,-0.003721,0.000156
3,-0.001353,0.000139,-0.065096,-0.057096,0.193164,0.113626,-0.000547,0.080283,-0.012290,-0.025152,...,-0.000016,0.059982,0.000064,0.014946,0.031229,0.009437,0.032536,0.000489,0.066428,0.000043
4,-0.003586,0.009182,0.021582,-0.030150,0.239499,-0.012419,0.011642,0.077223,0.002664,0.008873,...,-0.000005,0.033490,0.000052,0.029382,0.037159,0.011544,0.123791,-0.013921,0.024804,-0.000070
5,0.020626,-0.031499,0.493114,0.699624,0.064580,0.105436,0.297689,1.422237,0.022474,0.296479,...,0.000457,0.009697,0.002108,1.231089,0.401294,0.144115,0.026133,0.001027,0.014038,0.000404


In [872]:
#creating function to get recommendations
def svd_recommender(df_predict, user, umr, number_recomm):
    user_predicted_movies = df_predict.loc[user, :].sort_values(ascending = False)
    original_data = umr.loc[user, :].sort_values(ascending = False)
    already_rated = user_movies.loc[user, :].dropna()
    unrated = list(user_movies.loc[1, pd.isnull(user_movies.loc[user, :])].index)
    
    recommendations = df_predict.loc[user][unrated]
    
    recommendations = pd.DataFrame(recommendations.sort_values(ascending = False).index[:number_recomm])
    
    return recommendations, already_rated

What the above function does is it takes a user, looks at what movies (s)he's already predicted, ignores them and returns the top 10 recommended movies. It also returns the top movies the person has rated to compare the results.

In [873]:
#getting values for user no. 650
recommend_650, rated_650 = svd_recommender(predicted_df, 650, svd_movies, 10)

In [874]:
print('Top 10 Recommendations for User 650')
print(recommend_1)

Top 10 Recommendations for User 1
                                title
0             Schindler's List (1993)
1    Shawshank Redemption, The (1994)
2                    Apollo 13 (1995)
3          Usual Suspects, The (1995)
4                  Stand by Me (1986)
5                         Babe (1995)
6                Seven (Se7en) (1995)
7  Four Weddings and a Funeral (1994)
8                        Glory (1989)
9       Much Ado About Nothing (1993)


In [891]:
print('Best rated movies by the user')
rated_1.sort_values(ascending = False)[:10]

Best rated movies by the user


title
Maltese Falcon, The (1941)    5.0
Young Frankenstein (1974)     5.0
Casablanca (1942)             5.0
Citizen Kane (1941)           5.0
Vertigo (1958)                5.0
Star Wars (1977)              5.0
Great Race, The (1965)        5.0
Lawrence of Arabia (1962)     5.0
Princess Bride, The (1987)    5.0
North by Northwest (1959)     5.0
Name: 650, dtype: float64

**Upcoming - Recommender systems with Deep Learning**